# SUM and COUNT

In [1]:
library(tidyverse)
library(DBI)
library(getPass)
drv <- switch(Sys.info()['sysname'],
             Windows="PostgreSQL Unicode(x64)",
             Darwin="/usr/local/lib/psqlodbcw.so",
             Linux="PostgreSQL")
con <- dbConnect(
  odbc::odbc(),
  driver = drv,
  Server = "localhost",
  Database = "sqlzoo",
  UID = "postgres",
  PWD = getPass("Password?"),
  Port = 5432
)
options(repr.matrix.max.rows=20)

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.0     v purrr   0.3.4
v tibble  3.0.1     v dplyr   0.8.5
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



Password? ·········


## 1. Total world population

Show the total population of the world.

```
world(name, continent, area, population, gdp)
```

In [2]:
world <- dbReadTable(con, 'world')

In [3]:
world %>%
    summarise(popl=sum(population))

popl
<dbl>
7118632738


## 2. List of continents

List all the continents - just once each.

In [4]:
world %>% 
    distinct(continent)

continent
<chr>
Asia
Europe
Africa
Caribbean
South America
Eurasia
Oceania
North America


## 3. GDP of Africa

Give the total GDP of Africa

In [5]:
world %>%
    filter(continent=='Africa') %>% 
    summarise(gdp=sum(gdp, na.rm=TRUE))

gdp
<dbl>
1.811788e+12


## 4. Count the big countries

How many countries have an **area** of at least 1000000

In [6]:
world %>% 
    filter(area>=1e6) %>% 
    summarise(n=n())

n
<int>
29


## 5. Baltic states population

What is the total **population** of ('Estonia', 'Latvia', 'Lithuania')

In [7]:
world %>% 
    filter(name %in% c('Estonia', 'Latvia', 'Lithuania')) %>%
    summarise(popl=sum(population))

popl
<dbl>
6251750


## 6. Counting the countries of each continent

For each **continent** show the **continent** and number of countries.

> _Using GROUP BY and HAVING_   
> You may want to look at these examples: [Using GROUP BY and HAVING](https://sqlzoo.net/wiki/Using_GROUP_BY_and_HAVING.).

In [8]:
world %>% 
    group_by(continent) %>%
    summarise(n=n())

continent,n
<chr>,<int>
Africa,53
Asia,47
Caribbean,11
Eurasia,2
Europe,44
North America,11
Oceania,14
South America,13


## 7. Counting big countries in each continent

For each **continent** show the **continent** and number of countries with populations of at least 10 million.

In [9]:
world %>% 
    group_by(continent) %>% 
    filter(population>=1e7) %>%
    summarise(n=n())

continent,n
<chr>,<int>
Africa,29
Asia,26
Caribbean,2
Eurasia,1
Europe,14
North America,4
Oceania,1
South America,8


## 8. Counting big continents

List the continents that **have** a total population of at least 100 million.

In [10]:
world %>% 
    group_by(continent) %>% 
    summarise(popl=sum(population)) %>% 
    filter(popl>=1e8) %>% 
    select(continent)

continent
<chr>
Africa
Asia
Eurasia
Europe
North America
South America


In [11]:
dbDisconnect(con)